In [6]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __builtin__ import any as b_any

import math
import os
os.environ["CUDA_VISIBLE_DEVICES"]=""
import numpy as np
from PIL import Image

import tensorflow as tf

import configuration
import inference_wrapper
import sys
sys.path.insert(0, 'im2txt/inference_utils')
sys.path.insert(0, 'im2txt/ops')
import caption_generator
import image_processing
import vocabulary

In [7]:
np.random.seed(2)

current_dir = os.getcwd()
image_dir = os.path.join(current_dir, 'Jmemes')


checkpoint_path="trainlogIncNEW"
vocab_file ="vocab4.txt"
input_files ="Jmemes"         

tf.logging.set_verbosity(tf.logging.INFO)


In [ ]:
#configuration
config = configuration.ModelConfig()

g = tf.Graph()
with g.as_default():
    model = inference_wrapper.InferenceWrapper()
    restore_fn = model.build_graph_from_config(config,
                                               checkpoint_path)
g.finalize()


INFO:tensorflow:Building model.
About to decide if splitting
new_h Tensor("lstm/basic_lstm_cell/Mul_2:0", shape=(1, 512), dtype=float32)
new_state LSTMStateTuple(c=<tf.Tensor 'lstm/basic_lstm_cell/Add_1:0' shape=(1, 512) dtype=float32>, h=<tf.Tensor 'lstm/basic_lstm_cell/Mul_2:0' shape=(1, 512) dtype=float32>)
About to decide if splitting
It's splitting
gate_inputs0 Tensor("lstm/basic_lstm_cell/MatMul_1:0", shape=(1, 1, 2048), dtype=float32)
gate_inputs1 Tensor("lstm/basic_lstm_cell/MatMul_2:0", shape=(1, 1, 2048), dtype=float32)
<tf.Variable 'lstm/basic_lstm_cell/kernel:0' shape=(812, 2048) dtype=float32_ref>
gate_inputs Tensor("lstm/basic_lstm_cell/concat_2:0", shape=(1, 2, 2048), dtype=float32)
new_h Tensor("lstm/basic_lstm_cell/Mul_5:0", shape=(1, 2, 512), dtype=float32)
new_state LSTMStateTuple(c=<tf.Tensor 'lstm/basic_lstm_cell/Add_3:0' shape=(1, 2, 512) dtype=float32>, h=<tf.Tensor 'lstm/basic_lstm_cell/Mul_5:0' shape=(1, 2, 512) dtype=float32>)
lstm_outputs Tensor("lstm/basic_l

In [4]:
vocab = vocabulary.Vocabulary(vocab_file)

with open('Captions.txt','r') as f:
    data_captions = f.readlines()
data_captions = [s.lower() for s in data_captions]
  
with open('ordered_memes.txt','r') as f:
    ordered_memes = f.readlines()
ordered_memes = [meme.replace('\n','') for meme in ordered_memes]

INFO:tensorflow:Initializing vocabulary from file: vocab4.txt
INFO:tensorflow:Created vocabulary with 38521 words


In [5]:
np.random.seed(2)
filenames = [os.path.join(image_dir, f) for f in os.listdir(input_files) if f.endswith('.jpg')]
print(filenames)
tf.logging.info("Running caption generation on %d files matching %s",len(filenames), input_files)
with tf.Session(graph=g) as sess:
    # Load the model from checkpoint.
    restore_fn(sess)
    
    # Prepare the caption generator. Here we are implicitly using the default
    # beam search parameters. See caption_generator.py for a description of the
    # available beam search parameters.
    generator = caption_generator.CaptionGenerator(model, vocab, beam_size=config.beam_size)
    num_in_data_total = 0
    num_captions = 0
    for i,filename in enumerate(filenames):
      with tf.gfile.GFile(filename, "rb") as f:
        #image = f.read()
        image = Image.open(f)
        image = ((np.array(image.resize((299,299)))/255.0)-0.5)*2.0 
        
      print("Captions for image %s:" % os.path.basename(filenames[i]))
      num_in_data = 0
      for k in range(10):
          captions = generator.beam_search(sess, image)
            
          for i, caption in enumerate(captions):
            # Ignore begin and end words.
            sentence = [vocab.id_to_word(w) for w in caption.sentence[1:-1]]
            sentence = " ".join(sentence)
            in_data = 0
            if b_any(sentence in capt for capt in data_captions):
                in_data = 1
                num_in_data += 1
                num_in_data_total += 1
                num_captions += 1
            else:
                num_captions += 1
            print("  %d) %s (p=%f) [in data = %d]" % (i, sentence, math.exp(caption.logprob),in_data))
      print("number of captions in data = %d" % (num_in_data))
    print("(total number of captions in data = %d) percent in data = %f" % (num_in_data_total,(num_in_data_total/num_captions)))

['/Users/ALP/PycharmProjects/MemeProject/im2txt/Jmemes/sassy-gay-snape.jpg', '/Users/ALP/PycharmProjects/MemeProject/im2txt/Jmemes/NOSE.jpg', '/Users/ALP/PycharmProjects/MemeProject/im2txt/Jmemes/romneyhood.jpg', '/Users/ALP/PycharmProjects/MemeProject/im2txt/Jmemes/ridiculously-photogenic-metalhead-guy.jpg', '/Users/ALP/PycharmProjects/MemeProject/im2txt/Jmemes/one-does-not-simply.jpg', '/Users/ALP/PycharmProjects/MemeProject/im2txt/Jmemes/sad-dog.jpg', '/Users/ALP/PycharmProjects/MemeProject/im2txt/Jmemes/sudden-realization-ralph.jpg', '/Users/ALP/PycharmProjects/MemeProject/im2txt/Jmemes/y-u-no.jpg', '/Users/ALP/PycharmProjects/MemeProject/im2txt/Jmemes/stoner-stanley.jpg', '/Users/ALP/PycharmProjects/MemeProject/im2txt/Jmemes/stoner-dog.jpg', '/Users/ALP/PycharmProjects/MemeProject/im2txt/Jmemes/advice-dog.jpg']
INFO:tensorflow:Running caption generation on 11 files matching Jmemes
INFO:tensorflow:Loading model from checkpoint: trainlogIncNEW/model.ckpt-1000000
INFO:tensorflow:Rest

KeyboardInterrupt: 

In [ ]:
np.random.seed(2)
x = np.array([[np.ones([300])*0.5,np.ones([300])*0.5]])
print(x[0,0,15])
print(x[0,1,15])

In [ ]:
np.random.seed(2)

seq_embeddings = np.array([[np.ones([300])*0.5,np.ones([300])*0.5]])
state_feed = np.array([[np.ones([1024])*0.6,np.ones([1024])*0.6]])
print(seq_embeddings[0,0,:10])
print(seq_embeddings[0,1,:10])
print(state_feed[0,0,:10])
print(state_feed[0,1,:10])
image_feed = np.ones([299,299,3])*-0.2156862745
with tf.Session(graph=g) as sess:
    # Load the model from checkpoint.
    restore_fn(sess)
    softmax, state = sess.run(
        fetches=["softmax:0", "lstm/state:0"],
        #fetches=["lstm/initial_state:0"],
        feed_dict={#"image_feed:0": image_feed
            #"input_feed:0": input_feed,
            "lstm/state_feed:0": state_feed,
            "seq_embeddings:0": seq_embeddings,
            #"seq_embedding/embedding_map:0": self.embedding_map
        })
    print(state.shape)
    #print(softmax_output.shape)

In [ ]:
print(state[0,1,90:100])
print(state[0,0,90:100])

In [ ]:
print(initial_state_output[0][0][0])
print(initial_state_output[0][0][-1])

In [ ]:
state_output[0].shape

In [ ]:
np.savetxt('stateoutputTEST.txt',state_output[0])

In [ ]:
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1000px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [ ]:
show_graph(g)

In [ ]:
OLD_CHECKPOINT_FILE = "trainlogSGD/model.ckpt-600000"
NEW_CHECKPOINT_FILE = "trainlogSGDNEW/model.ckpt-600000"

import tensorflow as tf
vars_to_rename = {
    "lstm/basic_lstm_cell/weights": "lstm/basic_lstm_cell/kernel",
    "lstm/basic_lstm_cell/biases": "lstm/basic_lstm_cell/bias",
}
new_checkpoint_vars = {}
reader = tf.train.NewCheckpointReader(OLD_CHECKPOINT_FILE)
for old_name in reader.get_variable_to_shape_map():
  if old_name in vars_to_rename:
    new_name = vars_to_rename[old_name]
  else:
    new_name = old_name
  new_checkpoint_vars[new_name] = tf.Variable(reader.get_tensor(old_name))

init = tf.global_variables_initializer()
saver = tf.train.Saver(new_checkpoint_vars)

with tf.Session() as sess:
  sess.run(init)
  saver.save(sess, NEW_CHECKPOINT_FILE)